In [153]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import sys, traceback
import json
from urllib import parse

In [3]:
KW_CODE =0
KM_CODE =1
DJ_CODE=2
DS_CODE=3
DD_CODE=4
MJ_CODE=5
SY_CODE=6
SM_CODE=7
SK_CODE=8
SW_CODE=9
SS_CODE=10
HS_CODE=11

In [4]:
base_start_url_list = ['http://kupis.kw.ac.kr/',
'http://lib.kookmin.ac.kr/',
'http://library.daejin.ac.kr/',
'http://discover.duksung.ac.kr/',
'http://library.dongduk.ac.kr/',
'http://lib.mju.ac.kr/',
'http://lib.syu.ac.kr/',
'https://lib.smu.ac.kr/',
'http://library.skuniv.ac.kr/',
'http://lib.swu.ac.kr/',
'http://lib.sungshin.ac.kr/',
'http://hsel.hansung.ac.kr/']


In [213]:
stateDict = {'대출중': 0, '이용가능':1,'대출가능':1, '지정도서':2}

In [51]:
base_middle_url_list = ['search/caz/result?st=KWRD&commandType=advanced&si=1&q=',
'#/total-search?keyword=',
'',
'',
'',
'',
'',
'Search/?q=',
'',
'',
'',
'']

In [52]:
base_end_url_list = ['&b0=and&weight0=&si=2&q=&b1=and&weight1=&si=3&q=&weight2=&inc=TOTAL&_inc=on&_inc=on&_inc=on&_inc=on&_inc=on&_inc=on&lmt0=TOTAL&lmtsn=000000000003&lmtst=OR&rf=&rt=&range=000000000021&cpp=10&msc=100',
'',
'',
'',
'',
'',
'',
'',
'',
'',
'',
''
]


In [7]:
class LibraryLS:
    def __init__(self, sid):
        self.sid = sid
        self.loanStatusList = []
        self.errorMessage= ''
    

In [8]:
class LoanStatus:
    def __init__(self, title = '', RN = '', CN = '', POS = '', STATE = -1, RDD = '', BN = 0, errorMessage = ''):
        self.title = title
        self.RN= RN
        self.CN= CN
        self.POS= POS
        self.STATE= STATE
        self.RDD = RDD#'2222-12-31'
        self.BN = BN
        self.errorMessage = errorMessage
'''#STATE : 도서 상태,
#-1  에러
#1 대출가능
#0 대출중
#2 지정도서
# 대출불가 아직 생각 안함.
#RN : 등록번호
#CN : 청구기호
#POS : 위치
이런것들'''

'#STATE : 도서 상태,\n#-1  에러\n#1 대출가능\n#0 대출중\n#2 지정도서\n# 대출불가 아직 생각 안함.\n#RN : 등록번호\n#CN : 청구기호\n#POS : 위치\n이런것들'

In [9]:
def findSubStringByRegEx(text, sid, subStringType):
    regex = re.compile(regexDictList[sid][subStringType])
    matchObj = regex.search(text)
    subString = matchObj.group(1)
    return subString
    
    

In [10]:
def find_ISBN(text):
    regex = re.compile('\d\d\d\d\d\d\d\d\d\d\d\d\d')
    matchObj = regex.search(text)
    return matchObj.group(0)

In [11]:
def find_BN(text):
    if '예약' in text:
        regex = re.compile('대출중\s*\(\s+(\d+)명')
        matchObj = regex.search(text)
        num = int(matchObj.group(1))
        return num
    else :
        return 0

In [12]:
def find_RDD(text):
    regex = re.compile('\d\d\d\d\-\d\d\-\d\d')
    matchObj = regex.search(text)
    return matchObj.group(0)

In [13]:
def find_STATE_BN(text):
    if '대출중' in text:
        return 0, find_BN(text)
    elif '이용가능' in text:
        return 1,0
    elif '지정도서' in text:
        return 2,0
    else :
        return -1,0

In [212]:
def findState(text):
    try:
        STATE = stateDict[text]
    except KeyError:
        STATE = -1
    return STATE

In [229]:
'''def visitLink(ISBN, sid, title, bookLink):
    loanStatusList = []
    #= LoanStatus()
    try:
        #1. 접속.
        html = urlopen(base_start_url_list[sid]+bookLink)
        bsObject = BeautifulSoup(html, "html.parser")
        
        #1-1 ISBN 정보가 일치하지 않다면 바로 return!!!
        includedISBNString = str(bsObject.find('div', {'class':'col-md-10 detail-table-right'}).find_all('dl')[2].find('dd'))

        if ISBN != find_ISBN(includedISBNString):
            return loanStatusList
        #2. 대출 데이터를 차곡차곡저장.
        idx = 0
        for loanResult in bsObject.find('div', class_='sponge-guide-Box-table sponge-detail-table').find('tbody').find_all('tr'):
            errorMessage = ''
            content = loanResult.find_all('td')
            RN = content[0].text.strip()
            CN = content[1].text.strip()
            POS = content[2].text.strip()
            STATE_STRING = content[3].text.strip()
            try:
                RDD = find_RDD(str(content[4]))
            except AttributeError as e:
                RDD = ''
            STATE, BN = find_STATE_BN(STATE_STRING)
            loanStatusList.append(LoanStatus(title, RN, CN,POS,STATE, RDD ,BN, ''))
    except :
        pass
        #traceback.format_exc()
    finally:
        return loanStatusList'''

'def visitLink(ISBN, sid, title, bookLink):\n    loanStatusList = []\n    #= LoanStatus()\n    try:\n        #1. 접속.\n        html = urlopen(base_start_url_list[sid]+bookLink)\n        bsObject = BeautifulSoup(html, "html.parser")\n        \n        #1-1 ISBN 정보가 일치하지 않다면 바로 return!!!\n        includedISBNString = str(bsObject.find(\'div\', {\'class\':\'col-md-10 detail-table-right\'}).find_all(\'dl\')[2].find(\'dd\'))\n\n        if ISBN != find_ISBN(includedISBNString):\n            return loanStatusList\n        #2. 대출 데이터를 차곡차곡저장.\n        idx = 0\n        for loanResult in bsObject.find(\'div\', class_=\'sponge-guide-Box-table sponge-detail-table\').find(\'tbody\').find_all(\'tr\'):\n            errorMessage = \'\'\n            content = loanResult.find_all(\'td\')\n            RN = content[0].text.strip()\n            CN = content[1].text.strip()\n            POS = content[2].text.strip()\n            STATE_STRING = content[3].text.strip()\n            try:\n                RDD = 

In [228]:
def makeJsonDict(result):
    # result 의 타입은 LibraryLS
    jsonDict = {}
    jsonDict['sid'] = result.sid
    jsonDict['loanStatusList'] = []
    for item in result.loanStatusList:
        jsonDict['loanStatusList'].append(vars(item))
    jsonDict['errorMessage']=result.errorMessage
    return jsonDict

In [16]:
#아직 상명대만 함. 상명대 ISBN으로 검색예정.
def makeSearchUrl(ISBN, title, sid, flag):
    if flag ==1:
        searchUrl = base_start_url_list[sid]+base_middle_url_list[sid] + ISBN+ base_end_url_list[sid]
    else :
        # flag 2.
        searchUrl = base_start_url_list[sid]+base_middle_url_list[sid] + title+ base_end_url_list[sid]
    return searchUrl

In [242]:
#flag =1 iSBN, 0, title search by..
#함수 만들기.

#sid = int(sid)
def findLoanStatus(ISBN, title, sid):
    if sid != SM_CODE and sid != KW_CODE:
        result = LibraryLS(sid)
        result.errorMessage = 'Not developed...'
    else : 
        searchUrl = makeSearchUrl(ISBN, title, sid, 2)
        result = crawling(ISBN, title, sid, searchUrl)
    # json string 으로 만들기.
    jsonDict = makeJsonDict(result)
    return json.dumps(jsonDict)
    

In [18]:
'''test_string = ""
def crawling(ISBN, title, sid, searchUrl):
    libraryLoanStatus = LibraryLS(sid)
    try:
        #1. 접속.
        html = urlopen(searchUrl)
        bsObject = BeautifulSoup(html, "html.parser")
        #2. 링크를 차곡차곡저장.
        # 정규식과 beautifulsoap4을 이용하여 저장...!!
        bookLinklist =[]
        for bookResult in bsObject.find_all('div', class_='search-list-result'):
            content = bookResult.find('div', {'class','sponge-list-content'}).find('a')
            title = content.text.strip()
            link =  content.get('href')
            bookLinklist.append([title, link])

        #3. 해당 링크를 방문하여 리스트에 추가해라..!!
        for item in bookLinklist:
            libraryLoanStatus.loanStatusList += visitLink(ISBN, sid, item[0], item[1])
    except :
        libraryLoanStatus.errorMessage = traceback.format_exc()
        #libraryLoanStatus.errorMessage = 에러 traceback 에러메세지를 담는다.
    finally:
        return libraryLoanStatus
    '''

In [53]:
#searchUrl = 'http://kupis.kw.ac.kr/search/tot/result?st=KWRD&si=TOTAL&q=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EB%B6%84%EC%84%9D+%EC%A0%84%EB%AC%B8%EA%B0%80+%EA%B0%80%EC%9D%B4%EB%93%9C+%3D+The+guide+for+advanced+data+analytics+professional+%2F#'
ISBN = '9788988474839'
koreaTitle = '데이터 분석 전문가 가이드 = The guide for advanced data analytics professional'
sid = KW_CODE
title = parse.quote(koreaTitle)
searchUrl = makeSearchUrl(ISBN,title, sid,2)
#html = urlopen(searchUrl)
#bsObject = BeautifulSoup(html)

In [54]:
searchUrl

'http://kupis.kw.ac.kr/search/caz/result?st=KWRD&commandType=advanced&si=1&q=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D%20%EC%A0%84%EB%AC%B8%EA%B0%80%20%EA%B0%80%EC%9D%B4%EB%93%9C%20%3D%20The%20guide%20for%20advanced%20data%20analytics%20professional&b0=and&weight0=&si=2&q=&b1=and&weight1=&si=3&q=&weight2=&inc=TOTAL&_inc=on&_inc=on&_inc=on&_inc=on&_inc=on&_inc=on&lmt0=TOTAL&lmtsn=000000000003&lmtst=OR&rf=&rt=&range=000000000021&cpp=10&msc=100'

In [68]:
html = urlopen(searchUrl)
bsObject = BeautifulSoup(html, "html.parser")

In [69]:
bsObject


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>광운대학교 중앙도서관</title>
<link href="/image/ko/favicon.ico" rel="shortcut icon"/>
<link href="/style/ko/common.css" rel="stylesheet" type="text/css"/>
<link href="/style/ko/sub.css" rel="stylesheet" type="text/css"/>
<link href="/style/ko/guide.css" rel="stylesheet" type="text/css"/>
<script src="/js/common/jquery.js" type="text/javascript"></script>
<script src="/js/common/jquery.animate.clip.js" type="text/javascript"></script>
<script src="/script/ko/main.js" type="text/javascript"></script>
<script src="/script/ko/menu.js" type="text/javascript"></script>
<link href="/style/ko/standard.css" rel="stylesheet" type="text/css"/>
<link href="/style/ko/search/advanced.css" rel="stylesheet" type="text/css"/>
<link href="/style/ko/search/detail.css" rel="stylesheet" type="text/css"/>
<link hre

In [72]:
r1 = bsObject.find('div', {'id' : 'divSearchResult'})

In [73]:
r1

<div id="divSearchResult">
<div id="searchCnt">
<div style="float: left">
			
			
			총 <strong>2</strong>건 중 <strong>2</strong>건 출력
		</div>
<div style="text-align: right">
<strong>1/1</strong> 페이지
			
			
			<a href="/search/caz/excel?commandType=advanced&amp;lmtsn=000000000003&amp;cpp=10&amp;range=000000000021&amp;_inc=on&amp;_inc=on&amp;_inc=on&amp;_inc=on&amp;_inc=on&amp;_inc=on&amp;lmt0=TOTAL&amp;rf=&amp;si=1&amp;si=2&amp;si=3&amp;weight2=&amp;weight0=&amp;weight1=&amp;inc=TOTAL&amp;q=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EB%B6%84%EC%84%9D+%EC%A0%84%EB%AC%B8%EA%B0%80+%EA%B0%80%EC%9D%B4%EB%93%9C+%3D+The+guide+for+advanced+data+analytics+professional&amp;q=&amp;q=&amp;b0=and&amp;b1=and&amp;lmtst=OR&amp;rt=&amp;st=KWRD&amp;msc=100"><img alt="엑셀파일 출력" height="12" src="/image/ko/common/ico/excel.gif" title="엑셀파일 출력" width="12"/></a>
</div>
</div>
<div id="divSideInfo">
<div id="divFacetResult">
<div id="selectedFacet">
</div>
<div class="facetList" id="facetList">
<span class="black_12w">검색결과제한

In [175]:
r2 = r1.find_all('dl', {'class', 'briefDetail'})

In [176]:
type(r1)

bs4.element.Tag

In [177]:
len(r2)

2

In [178]:
type(r2[0])

bs4.element.Tag

In [195]:
r2

[<dl class="briefDetail">
 <dt class="skip">서명</dt>
 <dd class="searchTitle">
 <a href="/search/detail/CATCAZ000000438290?mainLink=/search/caz&amp;briefLink=/search/caz/result?commandType=advanced_A_lmtsn=000000000003_A_cpp=10_A_range=000000000021_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A_lmt0=TOTAL_A_rf=_A_si=1_A_si=2_A_si=3_A_weight2=_A_weight0=_A_weight1=_A_inc=TOTAL_A_q=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EB%B6%84%EC%84%9D+%EC%A0%84%EB%AC%B8%EA%B0%80+%EA%B0%80%EC%9D%B4%EB%93%9C+%3D+The+guide+for+advanced+data+analytics+professional_A_q=_A_q=_A_b0=and_A_b1=and_A_lmtst=OR_A_rt=_A_st=KWRD_A_msc=100">데이터 분석 전문가 가이드 = The guide for advanced data analytics professional / 개정판</a> 
 	                            
 	                                
 	                                
 	                                    <img alt="미리보기" height="13" id="preview_open_CATCAZ000000438290" onclick="preview('CATCAZ000000438290')" src="/image/ko/common/ico/advance.gif" style="cursor: po

In [179]:
r3 = r2[0].find('dd', {'class': 'searchTitle'}).find('a')

In [180]:
r3

<a href="/search/detail/CATCAZ000000438290?mainLink=/search/caz&amp;briefLink=/search/caz/result?commandType=advanced_A_lmtsn=000000000003_A_cpp=10_A_range=000000000021_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A_lmt0=TOTAL_A_rf=_A_si=1_A_si=2_A_si=3_A_weight2=_A_weight0=_A_weight1=_A_inc=TOTAL_A_q=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EB%B6%84%EC%84%9D+%EC%A0%84%EB%AC%B8%EA%B0%80+%EA%B0%80%EC%9D%B4%EB%93%9C+%3D+The+guide+for+advanced+data+analytics+professional_A_q=_A_q=_A_b0=and_A_b1=and_A_lmtst=OR_A_rt=_A_st=KWRD_A_msc=100">데이터 분석 전문가 가이드 = The guide for advanced data analytics professional / 개정판</a>

In [181]:
type(r3)

bs4.element.Tag

In [182]:
link = r3.get('href')
title = r3.text.strip()

In [183]:
print(link)
print(title)

/search/detail/CATCAZ000000438290?mainLink=/search/caz&briefLink=/search/caz/result?commandType=advanced_A_lmtsn=000000000003_A_cpp=10_A_range=000000000021_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A_lmt0=TOTAL_A_rf=_A_si=1_A_si=2_A_si=3_A_weight2=_A_weight0=_A_weight1=_A_inc=TOTAL_A_q=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EB%B6%84%EC%84%9D+%EC%A0%84%EB%AC%B8%EA%B0%80+%EA%B0%80%EC%9D%B4%EB%93%9C+%3D+The+guide+for+advanced+data+analytics+professional_A_q=_A_q=_A_b0=and_A_b1=and_A_lmtst=OR_A_rt=_A_st=KWRD_A_msc=100
데이터 분석 전문가 가이드 = The guide for advanced data analytics professional / 개정판


In [205]:
def makeBookResultList(bsObject, sid):
    if sid == KW_CODE:
        BookResultList = bsObject.find('div', {'id' : 'divSearchResult'}).find_all('dl', {'class': 'briefDetail'})
    elif sid == KM_CODE:
        pass
    elif sid == DJ_CODE:
        pass
    elif sid == DS_CODE:
        pass
    elif sid == DD_CODE:
        pass
    elif sid == MJ_CODE:
        pass
    elif sid == SY_CODE:
        pass
    elif sid == SM_CODE:
        BookResultList =  bsObject.find_all('div', class_='search-list-result')
    elif sid == SK_CODE:
        pass
    elif sid == SW_CODE:
        pass
    elif sid == SS_CODE:
        pass
    elif sid == HS_CODE:
        pass
    return BookResultList

In [185]:
bsObject


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>광운대학교 중앙도서관</title>
<link href="/image/ko/favicon.ico" rel="shortcut icon"/>
<link href="/style/ko/common.css" rel="stylesheet" type="text/css"/>
<link href="/style/ko/sub.css" rel="stylesheet" type="text/css"/>
<link href="/style/ko/guide.css" rel="stylesheet" type="text/css"/>
<script src="/js/common/jquery.js" type="text/javascript"></script>
<script src="/js/common/jquery.animate.clip.js" type="text/javascript"></script>
<script src="/script/ko/main.js" type="text/javascript"></script>
<script src="/script/ko/menu.js" type="text/javascript"></script>
<link href="/style/ko/standard.css" rel="stylesheet" type="text/css"/>
<link href="/style/ko/search/advanced.css" rel="stylesheet" type="text/css"/>
<link href="/style/ko/search/detail.css" rel="stylesheet" type="text/css"/>
<link hre

In [186]:
res4 = makeBookResultList(bsObject, KW_CODE)

In [187]:
len(res4)

2

In [188]:
type(bsObject)

bs4.BeautifulSoup

In [189]:
type(res4)

bs4.element.ResultSet

In [190]:
res4[0]

<dl class="briefDetail">
<dt class="skip">서명</dt>
<dd class="searchTitle">
<a href="/search/detail/CATCAZ000000438290?mainLink=/search/caz&amp;briefLink=/search/caz/result?commandType=advanced_A_lmtsn=000000000003_A_cpp=10_A_range=000000000021_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A_lmt0=TOTAL_A_rf=_A_si=1_A_si=2_A_si=3_A_weight2=_A_weight0=_A_weight1=_A_inc=TOTAL_A_q=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EB%B6%84%EC%84%9D+%EC%A0%84%EB%AC%B8%EA%B0%80+%EA%B0%80%EC%9D%B4%EB%93%9C+%3D+The+guide+for+advanced+data+analytics+professional_A_q=_A_q=_A_b0=and_A_b1=and_A_lmtst=OR_A_rt=_A_st=KWRD_A_msc=100">데이터 분석 전문가 가이드 = The guide for advanced data analytics professional / 개정판</a> 
	                            
	                                
	                                
	                                    <img alt="미리보기" height="13" id="preview_open_CATCAZ000000438290" onclick="preview('CATCAZ000000438290')" src="/image/ko/common/ico/advance.gif" style="cursor: pointer;" 

In [206]:
def makeContent(bookResult, sid):
    print(type(bookResult))
    if sid == KW_CODE:
        content = bookResult.find('dd', {'class': 'searchTitle'}).find('a')
    elif sid == KM_CODE:
        pass
    elif sid == DJ_CODE:
        pass
    elif sid == DS_CODE:
        pass
    elif sid == DD_CODE:
        pass
    elif sid == MJ_CODE:
        pass
    elif sid == SY_CODE:
        pass
    elif sid == SM_CODE:
        content =   bookResult.find('div', {'class','sponge-list-content'}).find('a')
    elif sid == SK_CODE:
        pass
    elif sid == SW_CODE:
        pass
    elif sid == SS_CODE:
        pass
    elif sid == HS_CODE:
        pass
    return content

In [192]:
'''def makeContent(bookResult, sid):
    return{
        KW_CODE: eval("bookResult.find('dd', {'class': 'searchTitle'}).find('a')"),
        SM_CODE: eval("bookResult.find('div', {'class','sponge-list-content'}).find('a')")
    }.get(sid, -1)'''

'def makeContent(bookResult, sid):\n    return{\n        KW_CODE: eval("bookResult.find(\'dd\', {\'class\': \'searchTitle\'}).find(\'a\')"),\n        SM_CODE: eval("bookResult.find(\'div\', {\'class\',\'sponge-list-content\'}).find(\'a\')")\n    }.get(sid, -1)'

In [201]:
makeContent(res4[0], sid)

<class 'bs4.element.Tag'>


<a href="/search/detail/CATCAZ000000438290?mainLink=/search/caz&amp;briefLink=/search/caz/result?commandType=advanced_A_lmtsn=000000000003_A_cpp=10_A_range=000000000021_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A_lmt0=TOTAL_A_rf=_A_si=1_A_si=2_A_si=3_A_weight2=_A_weight0=_A_weight1=_A_inc=TOTAL_A_q=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EB%B6%84%EC%84%9D+%EC%A0%84%EB%AC%B8%EA%B0%80+%EA%B0%80%EC%9D%B4%EB%93%9C+%3D+The+guide+for+advanced+data+analytics+professional_A_q=_A_q=_A_b0=and_A_b1=and_A_lmtst=OR_A_rt=_A_st=KWRD_A_msc=100">데이터 분석 전문가 가이드 = The guide for advanced data analytics professional / 개정판</a>

In [111]:
sid

0

In [ ]:
def 

In [245]:
def crawling(ISBN, title, sid, searchUrl):
    libraryLoanStatus = LibraryLS(sid)
    try:
        #1. 접속.
        html = urlopen(searchUrl)
        bsObject = BeautifulSoup(html, "html.parser")
        #2. 링크를 차곡차곡저장.
        # 정규식과 beautifulsoap4을 이용하여 저장...!!
        bookLinklist =[]
        for bookResult in makeBookResultList(bsObject, sid):
            content = makeContent(bookResult, sid)
            title = content.text.strip()
            link =  content.get('href')
            bookLinklist.append([title, link])
            
        print(bookLinklist)
        #3. 해당 링크를 방문하여 리스트에 추가해라..!!
        for item in bookLinklist:
            libraryLoanStatus.loanStatusList += visitLink(ISBN, sid, item[0], item[1])
    except:
        libraryLoanStatus.errorMessage = traceback.format_exc()
        #libraryLoanStatus.errorMessage = 에러 traceback 에러메세지를 담는다.
    finally:
        return libraryLoanStatus
    

In [209]:
#searchUrl = 'http://kupis.kw.ac.kr/search/tot/result?st=KWRD&si=TOTAL&q=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EB%B6%84%EC%84%9D+%EC%A0%84%EB%AC%B8%EA%B0%80+%EA%B0%80%EC%9D%B4%EB%93%9C+%3D+The+guide+for+advanced+data+analytics+professional+%2F#'
ISBN = '9788988474839'
koreaTitle = '데이터 분석 전문가 가이드 = The guide for advanced data analytics professional'
sid = KW_CODE
title = parse.quote(koreaTitle)
searchUrl = makeSearchUrl(ISBN,title, sid,2)
craw_res = crawling(ISBN, title, sid, searchUrl)

<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
[['데이터 분석 전문가 가이드 = The guide for advanced data analytics professional / 개정판', '/search/detail/CATCAZ000000438290?mainLink=/search/caz&briefLink=/search/caz/result?commandType=advanced_A_lmtsn=000000000003_A_cpp=10_A_range=000000000021_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A_lmt0=TOTAL_A_rf=_A_si=1_A_si=2_A_si=3_A_weight2=_A_weight0=_A_weight1=_A_inc=TOTAL_A_q=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EB%B6%84%EC%84%9D+%EC%A0%84%EB%AC%B8%EA%B0%80+%EA%B0%80%EC%9D%B4%EB%93%9C+%3D+The+guide+for+advanced+data+analytics+professional_A_q=_A_q=_A_b0=and_A_b1=and_A_lmtst=OR_A_rt=_A_st=KWRD_A_msc=100'], ['데이터 분석 전문가 가이드 = The guide for advanced data analytics professional [2014 ed.]', '/search/detail/CATCAZ000000389638?mainLink=/search/caz&briefLink=/search/caz/result?commandType=advanced_A_lmtsn=000000000003_A_cpp=10_A_range=000000000021_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A_lmt0=TOTAL_A_rf=_A_si=1_A_si=2_A_si

In [200]:
vars(craw_res)

{'sid': 0,
 'loanStatusList': [],
 'errorMessage': 'Traceback (most recent call last):\n  File "<ipython-input-100-08447b167766>", line 11, in crawling\n    content = makeContent(bookResult, sid)\n  File "<ipython-input-96-7ca200502045>", line 4, in makeContent\n    SM_CODE: bookResult.find(\'div\', {\'class\',\'sponge-list-content\'}).find(\'a\')\nAttributeError: \'NoneType\' object has no attribute \'find\'\n'}

In [104]:
searchUrl

'http://kupis.kw.ac.kr/search/caz/result?st=KWRD&commandType=advanced&si=1&q=%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D%20%EC%A0%84%EB%AC%B8%EA%B0%80%20%EA%B0%80%EC%9D%B4%EB%93%9C%20%3D%20The%20guide%20for%20advanced%20data%20analytics%20professional&b0=and&weight0=&si=2&q=&b1=and&weight1=&si=3&q=&weight2=&inc=TOTAL&_inc=on&_inc=on&_inc=on&_inc=on&_inc=on&_inc=on&lmt0=TOTAL&lmtsn=000000000003&lmtst=OR&rf=&rt=&range=000000000021&cpp=10&msc=100'

In [ ]:
c

In [237]:
def makeLoanResultList(bsObject, sid):
    if sid == KW_CODE:
        LoanResultList = bsObject.find('div', {'id': 'divHoldingInfo'}).find('table').find('tbody').find_all('tr')
    elif sid == KM_CODE:
        pass
    elif sid == DJ_CODE:
        pass
    elif sid == DS_CODE:
        pass
    elif sid == DD_CODE:
        pass
    elif sid == MJ_CODE:
        pass
    elif sid == SY_CODE:
        pass
    elif sid == SM_CODE:
        LoanResultList = bsObject.find('div', class_='sponge-guide-Box-table sponge-detail-table').find('tbody').find_all('tr')
    elif sid == SK_CODE:
        pass
    elif sid == SW_CODE:
        pass
    elif sid == SS_CODE:
        pass
    elif sid == HS_CODE:
        pass
    return LoanResultList

In [251]:
def getLoanStatus(title, loanResult, loanStatusList, sid):
    if sid == KW_CODE:
        errorMessage = ''
        content = loanResult.find_all('td')
        RN = content[1].text.strip()
        CN = content[2].text.strip()
        POS = content[3].text.strip()
        STATE_STRING = content[4].text.strip()
        RDD = content[5].text.strip()
        STATE = findState(STATE_STRING)
        BN = 0
    elif sid == KM_CODE:
        pass
    elif sid == DJ_CODE:
        pass
    elif sid == DS_CODE:
        pass
    elif sid == DD_CODE:
        pass
    elif sid == MJ_CODE:
        pass
    elif sid == SY_CODE:
        pass
    elif sid == SM_CODE:
        errorMessage = ''
        content = loanResult.find_all('td')
        RN = content[0].text.strip()
        CN = content[1].text.strip()
        POS = content[2].text.strip()
        STATE_STRING = content[3].text.strip()
        try:
            RDD = find_RDD(str(content[4]))
        except AttributeError as e:
            RDD = ''
        STATE, BN = find_STATE_BN(STATE_STRING)
    elif sid == SK_CODE:
        pass
    elif sid == SW_CODE:
        pass
    elif sid == SS_CODE:
        pass
    elif sid == HS_CODE:
        pass
    
    loanStatusList.append(LoanStatus(title, RN, CN,POS,STATE, RDD ,BN, ''))
    return

In [250]:
def visitLink(ISBN, sid, title, bookLink):
    loanStatusList = []
    #= LoanStatus()
    try:
        #1. 접속.
        print(base_start_url_list[sid]+bookLink)
        html = urlopen(base_start_url_list[sid]+bookLink)
        bsObject = BeautifulSoup(html, "html.parser")
        
        #1-1 ISBN 정보가 일치하지 않다면 바로 return!!!
        if isISBNCorrect(ISBN, bsObject, sid) == False:
            print('hi')
            return loanStatusList

        #2. 대출 데이터를 차곡차곡저장.
        idx = 0
        for loanResult in makeLoanResultList(bsObject, sid):
            print('hi2')
            getLoanStatus(title,loanResult, loanStatusList, sid)
    except :
        traceback.print_exc()
        #traceback.format_exc()
    finally:
        return loanStatusList

In [238]:
ISBN = '9788988474839'
sid = KW_CODE
title = '데이터 분석 전문가 가이드 = The guide for advanced data analytics professional / 개정판'
bookLink = '/search/detail/CATCAZ000000438290?mainLink=/search/caz&briefLink=/search/caz/result?commandType=advanced_A_lmtsn=000000000003_A_cpp=10_A_range=000000000021_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A_lmt0=TOTAL_A_rf=_A_si=1_A_si=2_A_si=3_A_weight2=_A_weight0=_A_weight1=_A_inc=TOTAL_A_q=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EB%B6%84%EC%84%9D+%EC%A0%84%EB%AC%B8%EA%B0%80+%EA%B0%80%EC%9D%B4%EB%93%9C+%3D+The+guide+for+advanced+data+analytics+professional_A_q=_A_q=_A_b0=and_A_b1=and_A_lmtst=OR_A_rt=_A_st=KWRD_A_msc=100'
resultfinal = visitLink(ISBN, sid, title, bookLink)

http://kupis.kw.ac.kr//search/detail/CATCAZ000000438290?mainLink=/search/caz&briefLink=/search/caz/result?commandType=advanced_A_lmtsn=000000000003_A_cpp=10_A_range=000000000021_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A__inc=on_A_lmt0=TOTAL_A_rf=_A_si=1_A_si=2_A_si=3_A_weight2=_A_weight0=_A_weight1=_A_inc=TOTAL_A_q=%EB%8D%B0%EC%9D%B4%ED%84%B0+%EB%B6%84%EC%84%9D+%EC%A0%84%EB%AC%B8%EA%B0%80+%EA%B0%80%EC%9D%B4%EB%93%9C+%3D+The+guide+for+advanced+data+analytics+professional_A_q=_A_q=_A_b0=and_A_b1=and_A_lmtst=OR_A_rt=_A_st=KWRD_A_msc=100
hi2
hi2
hi2


In [241]:
for item in resultfinal :
    print(vars(item))

{'title': '데이터 분석 전문가 가이드 = The guide for advanced data analytics professional / 개정판', 'RN': '744255', 'CN': '005.74 ㅎ155ㄷ-ㄴ 2016', 'POS': '1층 지정도서/', 'STATE': -1, 'RDD': '', 'BN': 0, 'errorMessage': ''}
{'title': '데이터 분석 전문가 가이드 = The guide for advanced data analytics professional / 개정판', 'RN': '744256', 'CN': '005.74 ㅎ155ㄷ-ㄴ 2016 c.2', 'POS': '1층-자연/일반(000-600)', 'STATE': 1, 'RDD': '', 'BN': 0, 'errorMessage': ''}
{'title': '데이터 분석 전문가 가이드 = The guide for advanced data analytics professional / 개정판', 'RN': '818160', 'CN': '005.74 ㅎ155ㄷ-ㄴ 2016 c.3', 'POS': '1층-자연/일반(000-600)', 'STATE': 1, 'RDD': '', 'BN': 0, 'errorMessage': ''}


In [215]:
def isISBNCorrect(ISBN, bsObject, sid):
    if sid == KW_CODE:
        searchedISBN = find_ISBN(str(bsObject.find('div', {'id': 'divProfile'})))
        if ISBN == searchedISBN:
            return True
        else :
            return False
    elif sid == DJ_CODE:
        pass
    elif sid == DS_CODE:
        pass
    elif sid == DD_CODE:
        pass
    elif sid == MJ_CODE:
        pass
    elif sid == SY_CODE:
        pass
    elif sid == SM_CODE:
        includedISBNString = str(bsObject.find('div', {'class':'col-md-10 detail-table-right'}).find_all('dl')[2].find('dd'))
        if ISBN == find_ISBN(includedISBNString):
            return True
        else :
            return False
    elif sid == SK_CODE:
        pass
    elif sid == SW_CODE:
        pass
    elif sid == SS_CODE:
        pass
    elif sid == HS_CODE:
        pass

In [252]:
ISBN = '9788988474839'
koreaTitle = '데이터 분석 전문가 가이드'
sid = KW_CODE
title = parse.quote(koreaTitle)
resultfinalfinal = findLoanStatus(ISBN, title, sid)

[]


In [253]:
resultfinalfinal

'{"sid": 0, "loanStatusList": [], "errorMessage": ""}'